## Ingesting PDF

In [2]:
%pip install --q unstructured langchain
%pip install --upgrade --quiet  "unstructured[all-docs]"


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [4]:
local_path = "AI_Anxiety_V4.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [5]:
# Preview first page
data[0].page_content

'See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/374092896\n\nAI Anxiety: A Comprehensive Analysis of Psychological Factors and Interventions\n\nArticle in SSRN Electronic Journal · September 2023\n\nDOI: 10.2139/ssrn.4573394\n\nCITATIONS 2\n\nREADS 1,749\n\n11 authors, including:\n\nJeff Kim\n\nShrinidhi Kadkol\n\nUniversity of Illinois at Chicago\n\nUniversity of Illinois at Chicago\n\n7 PUBLICATIONS 11 CITATIONS\n\n8 PUBLICATIONS 46 CITATIONS\n\nSEE PROFILE\n\nSEE PROFILE\n\nOlusola Ajilore\n\nUniversity of Illinois at Chicago\n\n254 PUBLICATIONS 3,934 CITATIONS\n\nSEE PROFILE\n\nAll content following this page was uploaded by Jeff Kim on 12 November 2023.\n\nThe user has requested enhancement of the downloaded file.\n\nAI ANXIETY: A COMPREHENSIVE ANALYSIS OF PSYCHOLOGICAL FACTORS AND INTERVENTIONS\n\nA PREPRINT\n\nJeff J.H. Kim Department of Biomedical Engineering University of Illinois College of Medicine Chicago, IL, U

## Vector Embeddings

In [6]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest ⠋ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest ⠙ pulling manifest 
pulling 970aa74c0a90... 100% ▕███████████████

In [7]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED               
mistral:latest         	61e88e884507	4.1 GB	21 minutes ago        	
nomic-embed-text:latest	0a109f422b47	274 MB	Less than a second ago	


In [8]:
%pip install --q chromadb
%pip install --q langchain-text-splitters


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [10]:
# Split and chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [11]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 14/14 [00:40<00:00,  2.90s/it]


## Retrieval

In [12]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [13]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [14]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [15]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [16]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
chain.invoke("What are potential solutions and mitigation measures to AI anxiety?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 10.46it/s]


' The article proposes several solutions and mitigation measures to address AI anxiety:\n\n1. AI-adapted clinical guidelines: Developing tailored diagnostic and therapeutic measures aimed at mitigating AI anxiety for different age groups is essential. These guidelines should be updated regularly to account for the evolving nature of AI technology.\n2. Continuous education: Updating school curricula to incorporate AI technology and its implications is crucial to help individuals adapt to the changing technological landscape. This includes demystifying AI and promoting responsible usage.\n3. Regulatory measures: Balancing AI innovation with societal adaptability requires regulatory measures. Engineering constraints and ethical design are essential components of responsible AI development.\n4. Mental health interventions: Leveraging AI in mental health interventions can help mitigate AI anxiety by providing accessible, personalized, and effective treatments for individuals experiencing an

In [18]:
# Delete all collections in the db
vector_db.delete_collection()